In [33]:
import talib as ta
from datetime import datetime
from datetime import timedelta
from bigtrader.constant import Direction, OrderType

STRATEGY_NAME = "STRATEGY_1m"
        
def initialize(context):
    """初始化"""
    print("initialize")  
#     context.symbol = instruments
#     context.count = 0
    context.my_instruments = context.get_conf_param("instruments")#从传入参数中获取需要交易的合约
    context.closetime_day = context.get_conf_param("closetime_day")#日内策略白盘平仓时间，一般14:58
    context.closetime_night = context.get_conf_param("closetime_night")#日内策略夜盘平仓时间，一般22:58，注意有些商品夜盘收盘时间不一样
    context.order_num = context.get_conf_param("order_num")#下单手数
    context.set_universe(context.my_instruments)#设置需要处理的合约
    # 设置每变动一格，增减的数量
    context.volume = 1
    # 储存前一个网格所处区间，用来和最新网格所处区间作比较
    context.last_grid = 0
#     # 如果是非交易时间段，等到上午9点或晚上21点再执行algo()
#     schedule(schedule_func = algo, date_rule = '1d', time_rule = '09:00:00')
#     schedule(schedule_func = algo, date_rule = '1d', time_rule = '21:00:00')

def before_trading(context, data):
    """盘前处理"""
    #print("before_trading")
    context.subscribe(context.my_instruments) #注册合约
    context.index = 0
    # 获取前一交易日的主力合约
    # 记录上一次交易时网格范围的变化情况（例如从4区到5区，记为4,5）
    context.grid_change_last = [0,0]
    # 以前一日的收盘价为中枢价格
    context.center = data.history(context.my_instruments,["close"],1,"1d").iat[0,0]

    
def handle_data(context, data):
    """Bar行情推送"""
    cur_date =  data.current_dt
    cur_hm = cur_date.strftime('%H:%M') #time           
    # 分别获取多头持仓，和空头持仓
    position_long = context.get_position(context.my_instruments, Direction.LONG)
    position_short = context.get_position(context.my_instruments, Direction.SHORT)
    # 获取当前价格
    price = data.current(context.my_instruments, "close")
    #部分品种夜盘收盘时间不一样，此时间表示指定的尾盘平仓时间往后偏移30分钟，这段时间内不能开新仓，只能平仓。给30分钟是为了足够的冗余
    closetime_nightshift = (datetime.strptime(context.closetime_night,'%H:%M') + timedelta(minutes = 30)).strftime('%H:%M')  
    #尾盘平仓
    if((cur_hm>=context.closetime_day and cur_hm<="15:00") or (cur_hm>=context.closetime_night and cur_hm<=closetime_nightshift)):
        if(position_long.current_qty != 0):
            rv = context.sell_close(context.my_instruments, position_long.avail_qty, price, order_type=OrderType.MARKET)
            msg = str(data.current_dt) +  " 尾盘平多 for " + context.my_instruments + " 最新价=" + str(price) + " rv:" + str(rv)
            context.write_log(msg, stdout=1) #输出关键日志
        if(position_short.current_qty != 0):
            rv = context.buy_close(context.my_instruments, position_short.avail_qty, price, order_type=OrderType.MARKET)
            msg = str(data.current_dt) +  " 尾盘平空 for " + context.my_instruments + " 最新价=" + str(price) + " rv:" + str(rv)
            context.write_log(msg, stdout=1) #输出关键日志
        #尾盘不开新仓，直接返回
        return
    
    # 设置网格和当前价格所处的网格区域
    context.band = np.array([0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1, 1.01, 1.02, 1.03, 1.04, 1.05, 1.06, 1.07]) * context.center
    grid = pd.cut([price], context.band, labels=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14])[0]
    # 如果价格超出网格设置范围，则提示调节网格宽度和数量
    if np.isnan(grid):
        print('价格波动超过网格范围，可适当调节网格宽度和数量')
    # 如果新的价格所处网格区间和前一个价格所处的网格区间不同，说明触碰到了网格线，需要进行交易
    # 如果新网格大于前一天的网格，做空或平多
    if context.last_grid < grid:
        # 记录新旧格子范围（按照大小排序）
        grid_change_new = [context.last_grid,grid]
        # 几种例外：
        # 当last_grid = 0 时是初始阶段，不构成信号
        # 如果此时grid = 3，说明当前价格仅在开盘价之下的3区域中，没有突破网格线
        # 如果此时grid = 4，说明当前价格仅在开盘价之上的4区域中，没有突破网格线
        if context.last_grid == 0:
            context.last_grid = grid
            return
        if context.last_grid != 0:
            # 如果前一次开仓是4-5，这一次是5-4，算是没有突破，不成交
            if grid_change_new != context.grid_change_last:
                # 更新前一次的数据
                context.last_grid = grid
                context.grid_change_last = grid_change_new
                # 如果有多仓，平多
                if position_long.current_qty != 0:
                    rv = context.sell_close(context.my_instruments,context.order_num, price, order_type=OrderType.MARKET)
                    msg = str(data.current_dt) +  " 平多 for " + context.my_instruments + " 最新价=" + str(price) + " rv:" + str(rv)
                    context.write_log(msg, stdout=1) #输出关键日志
                # 否则，做空
                if not position_long.current_qty != 0:
                    rv = context.sell_open(context.my_instruments, context.order_num, price, order_type=OrderType.MARKET)
                    msg = str(data.current_dt) +  " 开空 for " + context.my_instruments + " 最新价=" + str(price) + " rv:" + str(rv)
                    context.write_log(msg, stdout=1) #输出关键日志
                    
        # 如果新网格小于前一天的网格，做多或平空
        if context.last_grid > grid:
            # 记录新旧格子范围（按照大小排序）
            grid_change_new = [grid,context.last_grid]
            # 几种例外：
            # 当last_grid = 0 时是初始阶段，不构成信号
            # 如果此时grid = 3，说明当前价格仅在开盘价之下的3区域中，没有突破网格线
            # 如果此时grid = 4，说明当前价格仅在开盘价之上的4区域中，没有突破网格线
            if context.last_grid == 0:
                context.last_grid = grid
                return
            if context.last_grid != 0:
                # 如果前一次开仓是4-5，这一次是5-4，算是没有突破，不成交
                if grid_change_new != context.grid_change_last:
                    # 更新前一次的数据
                    context.last_grid = grid
                    context.grid_change_last = grid_change_new
                    # 如果有空仓，平空
                    if position_short.current_qty != 0:
                        rv = context.buy_close(context.my_instruments, context.order_num, price, order_type=OrderType.MARKET)
                        msg = str(data.current_dt) +  " 平空 for " + context.my_instruments + " 最新价=" + str(price) + " rv:" + str(rv)
                        context.write_log(msg, stdout=1) #输出关键日志
                    # 否则，做多
                    if not position_short.current_qty != 0:
                        rv = context.buy_open(context.my_instruments, context.order_num, price, order_type=OrderType.MARKET)
                        msg = str(data.current_dt) +  " 开多 for " + context.my_instruments + " 最新价=" + str(price) + " rv:" + str(rv)
                        context.write_log(msg, stdout=1) #输出关键日志
                        
    # 设计一个止损条件：当持仓量达到10手，全部平仓
    if position_long.current_qty == 10 or position_long.current_qty == 10:
        if(position_long.current_qty != 0):
            rv = context.sell_close(context.my_instruments, position_long.avail_qty, price, order_type=OrderType.MARKET)
            msg = str(data.current_dt) +  " 止损平多 for " + context.my_instruments + " 最新价=" + str(price) + " rv:" + str(rv)
            context.write_log(msg, stdout=1) #输出关键日志
        if(position_short.current_qty != 0):
            rv = context.buy_close(context.my_instruments, position_short.avail_qty, price, order_type=OrderType.MARKET)
            msg = str(data.current_dt) +  " 止损平空 for " + context.my_instruments + " 最新价=" + str(price) + " rv:" + str(rv)
            context.write_log(msg, stdout=1) #输出关键日志
        print('触发止损，全部平仓')



            
def handle_order(context, order):
    """委托回报推送"""
    msg = "handle_order:" + order.log_str()
    context.write_log(msg, stdout=1) 
         
def handle_trade(context, trade):
    """成交回报推送"""
    msg = "handle_trade:" + trade.log_str()
    context.write_log(msg, stdout=1) 
    # 分别获取多头持仓，和空头持仓
    position_long = context.get_position(trade.symbol, Direction.LONG)
    position_short = context.get_position(trade.symbol, Direction.SHORT)
    msg = "当前多头持仓:"+ str(position_long) + "当前空头持仓:"+ str(position_short)
    context.write_log(msg, stdout=1) 

instruments = "RB2110.SHF" #["RB2110.SHF"] #["RB2110.SHF"] [I2105.DCE"] ['RB']# RB2101.SHF 如果想用主连合约，可以出入合约简码，例如RB,I,CF

#需要交易者传入的参数
strategy_setting = [
    {
        "instruments": instruments,
        "order_num": 1,
        "closetime_day": "14:58",
        "closetime_night": "22:58"
    }    
    
]
start_date = "2021-01-06"
end_date = "2021-05-12"
md = M.hfbacktest.v1(start_date=start_date,
                     end_date=end_date,
                     instruments=[instruments], #只传入一个合约便于策略逻辑展示
                     capital_base=100000,
                     product_type=Product.FUTURE,
                     frequency=Frequency.MINUTE,
                     initialize=initialize,
                     before_trading_start=before_trading,
                     handle_data=handle_data,
                     handle_order=handle_order,
                     handle_trade=handle_trade,
                     plot_charts=True,
                     volume_limit=1.0,
                     disable_cache=0,
                     show_debug_info=1,
                     strategy_setting=strategy_setting,
                     slippage_type=SlippageType.FIXED,#滑点固定模式
                     slippage_value=1.0,#买卖双向各1个滑点
                     m_deps=np.random.rand())



[2021-05-25 16:32:54.929708] INFO: moduleinvoker: hfbacktest.v1 开始运行..

[2021-05-25 16:32:55.137945] INFO: hfbacktest: passed-in daily_data_ds:None

[2021-05-25 16:32:55.140062] INFO: hfbacktest: passed-in minute_data_ds:None

[2021-05-25 16:32:55.145960] INFO: hfbacktest: passed-in tick_data_ds:None

[2021-05-25 16:32:55.147409] INFO: hfbacktest: passed-in each_data_ds:None

...

2021-05-25 16:33:51.912967 strategy_20210525(bkt999,): handle_order:[bkt999,8,RB2110.SHF,short,open,0,1,5611.0,pending,09:28:00,8] 
2021-05-25 16:33:51.913726 strategy_20210525(bkt999,): 2021-05-06 09:28:00 开空 for RB2110.SHF 最新价=5611.0 rv:0 
2021-05-25 16:33:51.916894 strategy_20210525(bkt999,): handle_order:[bkt999,8,RB2110.SHF,short,open,1,1,5611.0,filled,09:28:00,8] 
2021-05-25 16:33:51.917995 strategy_20210525(bkt999,): handle_trade:[bkt999,8,RB2110.SHF,short,open,1,5609.0,SHFE.8,09:29:00.000000] 
2021-05-25 16:33:51.918123 strategy_20210525(bkt999,): 当前多头持仓:Position(bkt999,RB2110.SHF,long,current_qty:0,avail_qty:0,cost_price:0.0,last_price:0.0)当前空头持仓:Position(bkt999,RB2110.SHF,short,current_qty:1,avail_qty:1,cost_price:5609.0,last_price:5609.0) 


2021-05-25 16:33:52.211340 strategy_20210525(bkt999,): handle_order:[bkt999,9,RB2110.SHF,short,open,0,1,5663.0,pending,13:58:00,9] 
2021-05-25 16:33:52.211727 strategy_20210525(bkt999,): 2021-05-06 13:58:00 开空 for RB2110.SHF 最新价=5663.0 rv:0 
2021-05-25 16:33:52.213723 strategy_20210525(bkt999,): handle_order:[bkt999,9,RB2110.SHF,short,open,1,1,5663.0,filled,13:58:00,9] 
2021-05-25 16:33:52.214570 strategy_20210525(bkt999,): handle_trade:[bkt999,9,RB2110.SHF,short,open,1,5663.0,SHFE.9,13:59:00.000000] 
2021-05-25 16:33:52.214684 strategy_20210525(bkt999,): 当前多头持仓:Position(bkt999,RB2110.SHF,long,current_qty:0,avail_qty:0,cost_price:0.0,last_price:0.0)当前空头持仓:Position(bkt999,RB2110.SHF,short,current_qty:2,avail_qty:2,cost_price:5636.0,last_price:5663.0) 
2021-05-25 16:33:52.357704 strategy_20210525(bkt999,): handle_order:[bkt999,10,RB2110.SHF,long,close_today,0,2,5662.0,pending,14:58:00,10] 
2021-05-25 16:33:52.358244 strategy_20210525(bkt999,): 2021-05-06 14:58:00 尾盘平空 for RB2110.SHF 最新价=

2021-05-25 16:33:55.476507 run_backtest() finished! time cost 59.573s! 


[2021-05-25 16:33:56.631687] INFO: hfbacktest: backtest done, raw_perf_ds:DataSource(9de72b04c4724f7ca9178bf9eb62b88eT)

[2021-05-25 16:33:57.324572] INFO: moduleinvoker: hfbacktest.v1 运行完成[62.394888s].